## Testing different architectures

##### Loss function -> Cross entropy
##### Optimizer -> SGD

In [1]:
import torch.nn as nn
from classifier import (run_classifier, getting_data_batches, getting_total_batch_sizes, 
                        evaluating_classifier, CustomCNN, freeze_top_layers)

In [2]:
epochs = 5
batch_size = 8
train_dir = '../datasets/brain_tumor/train'
test_dir = '../datasets/brain_tumor/test'
data_batches = getting_data_batches(train_dir, test_dir, batch_size)
total_batch_sizes = getting_total_batch_sizes(data_batches['train'], data_batches['test'])

## Training a custom CNN

In [3]:
custom_cnn = CustomCNN(num_classes=4)
print('Custom CNN: {}'.format(custom_cnn))

Custom CNN: CustomCNN(
  (conv1): Conv2d(3, 32, kernel_size=(4, 4), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2304, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)


In [4]:
run_classifier(custom_cnn, epochs, batch_size, data_batches,   '../models/Custom_CNN.pth')
# evaluating_classifier(custom_cnn, data_batches, model_path='./models/Custom_CNN.pth')

Total batch sizes: {'train': 714, 'test': 164}
Epoch 0/4
----------


C:\Users\joseg\miniconda3\Lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


train Loss: 4.4281 Acc: 0.7913
test Loss: 3.1734 Acc: 0.8422
Epoch 1/4
----------
train Loss: 2.2315 Acc: 0.8964
test Loss: 2.4804 Acc: 0.8712
Epoch 2/4
----------
train Loss: 1.5208 Acc: 0.9298
test Loss: 2.1837 Acc: 0.8910
Epoch 3/4
----------
train Loss: 1.0178 Acc: 0.9531
test Loss: 0.9576 Acc: 0.9543
Epoch 4/4
----------
train Loss: 0.4420 Acc: 0.9830
test Loss: 0.7193 Acc: 0.9688
Training complete
Best val Acc: 0.968750
Saving model to ./models/Custom_CNN.pth


D:\joseg\Documents\Projects_practice\courses\Deep_Learning_Pytorch\transfer_learning\training_utils.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return model.load_st

Accuracy of the model on the test images: 96.9488939740656%


### Training VGG16 model, freeze top layers

In [8]:
from torchvision import models
vgg16_model = models.vgg16(weights=True)

In [9]:
def count_parameters(model):
    # calculate the top number of layers that the model has
    count = 0 
    for child in model.children():
        count+=1
    return count
count_parameters(vgg16_model)

3

In [ ]:
# VGG16 freeze top layers
vgg16_model = freeze_top_layers(vgg16_model, 1)
# replace the last layer
num_ftrs = vgg16_model.classifier[6].in_features
vgg16_model.fc = nn.Linear(num_ftrs, 4)
print('VGG16_freezing top layers: {}'.format(vgg16_model))

In [11]:
saving_path_VGG16 = '../models/VGG16_freeze_top_layers.pth'
run_classifier(vgg16_model, epochs, batch_size, data_batches, saving_path=saving_path_VGG16)
# evaluating_classifier(custom_cnn, data_batches, model_path = saving_path)

VGG16_freezing first 2 layers: VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, 

### Training Resnet18 model, freeze top layers

In [ ]:
resnet18_model = models.resnet18(pretrained=True)
count_parameters(resnet18_model)

In [ ]:
# Resnet18 freeze top layers
resnet18_model = freeze_top_layers(resnet18_model, 6)
# replace the last layer
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, 4)

In [ ]:
print('Resnet18_freezing top layers: {}'.format(resnet18_model))
saving_path_resnet = '../models/Resnet18_freeze_top_layers.pth'
run_classifier(resnet18_model, epochs, batch_size, data_batches, saving_path=saving_path_resnet)
# evaluating_classifier(custom_cnn, data_batches, model_path = saving_path)